# Data Links

This lists usefull Data sources : 

- UCI Machine Learning repository : https://archive.ics.uci.edu/ml/index.php
- UEA & UCR Time Series Classification Repository : https://timeseriesclassification.com/dataset.php

# Usefull tips

- Downsampling and Upsampling can be usefull to adapt data
- Using Interpolation technique carefully
- Smoothing can be usefull to use data for prediction (ex: pd.ewm performs exponential weighted smoothing : recent data have greater weight)
- Carefull know what the timestamps used mean (upload or data acquisition..) not to add lookahead in data

# Exploratory analysis

- Use Histogram, plots
- Statistical summary tables 
- Correlation tables 
- plot with diff to remove trends and time correlations

In [ ]:
import pandas as pd
from matplotlib import pyplot
from tslearn.clustering import TimeSeriesKMeans
from tsfresh import extract_features, extract_relevant_features
from tsfresh.examples.robot_execution_failures import load_robot_execution_failures, download_robot_execution_failures
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.varmax import VARMAX
from pmdarima.arima import auto_arima
import statsmodels.api as sm
from pandas import Series
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.ar_model import AutoReg
import warnings
warnings.filterwarnings('ignore')
download_robot_execution_failures()
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt 
%pylab inline
plt.style.use('dark_background')
import plotly.io as pio
from ds_toolbox.graphs import plot_evolution, plot_hist, plot_xy
plt.figure(figsize(20,10))
pd.options.plotting.backend = "plotly"
pio.templates.default = 'plotly_dark'
plt.style.use("dark_background")
dg = pd.read_csv('data_general/Daily_Demand_Forecasting_Orders.csv', sep=';')
# Data Daily total female births in California, 1959
df = pd.read_csv('data_general/daily-total-female-births-in-cal.csv', sep=',').iloc[0:365]
df.columns = ['date', 'female_births']
df.index= pd.to_datetime(df['date'])
df = plot_hist(df=df, keys=['female_births'], kernel_density='gaussian')

## Stationarity

A time series is stationarity if for any lags the distribution of values is equal <=> Strong Stationarity

Possible to transform data to make them stationarity (by differenciation, logarithm, squared) : but important to keep in mind the meaning of such transformation about data Informations.

In [ ]:
df['moving_mean'] = df.expanding(min_periods=2).mean()
df['moving_std'] = df['female_births'].expanding(min_periods=2).std()
df['EMA'] = df['female_births'].ewm(span=40,adjust=False).mean()
df = plot_evolution(df=df, keys=['female_births', 'moving_mean', 'moving_std', 'EMA'], title='Female Births in California 1959')

In [ ]:
print('Dickey-Fuller criteria: p=', str(sm.tsa.stattools.adfuller(df['female_births'])[1]))

## Auto-Correlation

In [ ]:
df['auto_corr'] = [df['female_births'].autocorr(lag=i) for i in range(len(df))]
df=plot_evolution(df=df.iloc[:len(df) -10], keys=['auto_corr'], title='Auto correlation functions')

In [ ]:
#plt.figure(figsize(30,10))
plot = plot_pacf(df['female_births'], lags=50)

## Spirious Correlation vs Cointegration

- Spirous Correlation are very high value of correlation without any reason (it could happend when there is a huge trend)
- Cointegration is high correlation due to real relation. Ex: Drunk Pedestrian with dog

## Residual Seasonal Trend Decomposition

In [ ]:
plt.figure(figsize(20,10))
plot = sm.tsa.seasonal_decompose(df['female_births']).plot()

# Statistical Models for Time Series

Ordinary least squares linear regression can be applied to time series data with following assumuptions : 

1. Assumptions with respect to behavior of time series
- time series has a linear response to its predictors (might be avoid this using polynomial features)
- No input variable is constant over time or perfectly correlated with another input variable : Independent variables to account for the temporal dimension of the data

2. Assumptions with respect to the error
- For each point in time the expected value of the error given all explanatory variables for all time periods is 0 : closed to impossible in practice 
- Error at any given time period is uncorrelated with inputs at any time periof in the past or future.
- Variance of error is independant of time

## Auto-Regressive Models

Intuition : Past predicts the future
$$y_t = b_0 + b_1 \times y_{t-1} + e _t$$
$e_t$ : variable term with mean at 0 and constant variance

AR(p) model Description : 
$$ y_t = b_0 + b_1 \times y_{t-1}  + b_2 \times y_{t-2}  + b_3 \times y_{t-3} + ... +  b_p \times y_{t-p} +  e _t $$

Ex : Ar(1)
$$ E(y_t) = \mu = \frac{b_0}{1-b_1}$$
$$ var(y_t) = \frac{var(e_t)}{1-b_1^2}$$

Which value for p ?

1. Use of Partial Autocorellation function and detect the first lag where it mets 5% threshold
2. Use of Akaike information Criterion : 
$$ AIC = 2k - 2lnL$$
$k$ : number of parameters
$L$ : likelihood of the model

In [ ]:
dljung = plot_evolution(df=acorr_ljungbox(df['female_births'].values, model_df=1, return_df=True), keys=['lb_pvalue'], title='Result of Ljung-Box for differents lags')

In [ ]:
dg = plot_evolution(df=dg, keys=['Target (Total orders)'], x_axis_name='index')

In [ ]:
plt.figure(figsize(30,10))

plot = plot_pacf(dg['Target (Total orders)'], lags=29)

In [ ]:
model1 = AutoReg(dg['Target (Total orders)'].values, lags=1).fit()
model2 = AutoReg(dg['Target (Total orders)'].values, lags=2).fit()
model3 = AutoReg(dg['Target (Total orders)'].values, lags=3).fit()
model4 = AutoReg(dg['Target (Total orders)'].values, lags=4).fit()
model5 = AutoReg(dg['Target (Total orders)'].values, lags=5).fit()

dg['prediction_1'] = model1.predict(start=0, end=len(dg), dynamic=False)
dg['prediction_2']=  [np.nan] + [item for item in model2.predict(start=0, end=len(dg), dynamic=False)]
dg['prediction_3']=  [np.nan,np.nan] + [item for item in model3.predict(start=0, end=len(dg), dynamic=False)]
dg['prediction_4']=  [np.nan,np.nan,np.nan] + [item for item in model4.predict(start=0, end=len(dg), dynamic=False)]
dg['prediction_5']=  [np.nan,np.nan,np.nan,np.nan] + [item for item in model5.predict(start=0, end=len(dg), dynamic=False)]
dg =  plot_evolution(df=dg, keys=['Target (Total orders)', 'prediction_1','prediction_2','prediction_3','prediction_4', 'prediction_5'], x_axis_name='index')
# make prediction
# yhat = model_fit.predict(dg['Target (Total orders)'].values[1:])
# yhat

## Moving Average Model

The value of each point in time is a function of the recent past value "error" terms (each of which is independant from the others)

$$ y_t = \mu + e_t + \theta_1 \times e_{t-1} + \theta_2 \times e_{t-2} + ... + \theta_q \times e_{t-q}$$

Concept : many independent events at different past times affect current value of the process

In [ ]:
dg['auto_corr'] = [dg['Banking orders (2)'].autocorr(lag=i) for i in range(len(dg))]
#from ds_toolbox.graphs import add_horizontal_line_trace
dg=plot_evolution(
    df=dg, keys=['auto_corr'],
    title='Auto correlation modes',
    modes=['markers'],
    bandwith={'up_value': 0.2, 'down_value':-0.2}
)

In [ ]:
# MA example
# fit model
MA1 = ARIMA(dg['Banking orders (2)'], order=(0,0, 1)).fit()
MA2 = ARIMA(dg['Banking orders (2)'], order=(0,0, 2)).fit()
# make prediction
dg['prediction_ma1']=  [item for item in MA1.predict(start=0, end=len(dg) - 1, dynamic=False)]
dg['prediction_ma2']=  [item for item in MA2.predict(start=0, end=len(dg) - 1, dynamic=False)]
dg = plot_evolution(df=dg, keys=['Banking orders (2)', 'prediction_ma1', 'prediction_ma2'])

## Autoregressive Integrated Moving Average Models

Combination of AM and MA models with acount of differencing to remove trends and rendering a time series stationary.

$$ y_t = \phi_1 \times y_{t-1} + ... + \phi_p \times y_{t-p} + e_t +  \theta_1 \times e_{t-1}+ ... + \theta_q \times e_{t-q}$$

The 'I' term in ARIMA stands for integrated which refers to how many times the modeled time series must be differenced to produce stationarity

In [ ]:
def evaluate_arima_model(X, arima_order):
    # prepare training dataset
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    try:
        error = mean_squared_error(test, predictions)
    except:
        error = np.inf
    return error

def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    mse = evaluate_arima_model(dataset, order)
                    if mse < best_score:
                        best_score, best_cfg = mse, order
                    print('ARIMA%s MSE=%.3f' % (order,mse))
                except:
                    continue
    print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))
    

In [ ]:
evaluate_models(dg['Banking orders (2)'].values, [0,1,2,3,4,5], [0,1,2], [0,1,2,3,4,5])

In [ ]:
arima = auto_arima(
    dg['Banking orders (2)'].values,
    start_p=1, start_q=1,
   max_p=5, max_q=5, m=12,
   start_P=0, seasonal=False,
   d=1, D=2, trace=True,
   error_action='ignore',  
   suppress_warnings=True, 
   stepwise=True
)
print(arima.aic())

In [ ]:
dg['prediction_autoarima'] = arima.predict_in_sample()
dg = plot_evolution(df=dg, keys=['Banking orders (2)', 'prediction_ma1', 'prediction_ma2', 'prediction_autoarima'])

## VAR and VARMA

It's a multivariate version of Autoregression : each variable has it's own auto regressive equation.
In matrix notation : 
$$y_t = \phi_0 + \phi_1 * y_{t-1} + \phi_2 * y_{t-2}$$

In [ ]:
model1 = VARMAX(dg[['Target (Total orders)', 'Banking orders (2)']].values).fit()
dg['target_varmax'] = [item[0] for item in model1.predict()]
dg['banking_varmax'] =[item[1] for item in model1.predict()]
dg = plot_evolution(df=dg, keys=['Banking orders (2)',  'banking_varmax'])

## Seasonal ARIMA

A seasonal ARIMA (SARIMA) model assumes multiplicative seasonality. 

ARIMA (p,d,q) * (P,D,Q)m

The model postulates that the seasonal behavior itself can be thought of as an ARIMA process with m specifying the number of time steps per seasonal cycle.

## Usefull Tips

- When trying to fit a AR model, it's usefull to perform  ACF on residual to see if we miss term. Performing it on series directly isn't efficient for AR model
- Using Ljung-Box test can confirm previous results
- ACF usefull to estimate all the interesting terms
- Model selection : If ACF behavior Falls of slowly and if PACF sharp drop after lag = p => AR best guess IF ACF sharp drop after lag=q and fall off slowly then MA, if no sharp cutoff anywere : ARMA
- General method : 
1. Determine wether AR MA or ARMA
2. Test by increasing order and inspect residual : if large pacf then increase autoregressive

## Pros and Cons

Advantages of Statistical Methods for Time Series : 
- models are simple and transparent
- good if only small data available
- practical methods to perform fit

Disadvantages : 
- performance of these models don't realy improove with more data
- Puts the focus on point estimates of the mean value of a distribution rather than on the distrbution.
- Will do poor job describing data where nonlinear relationships are dominant

# State Space Models for Time Series

Inspired from real-word usecases motivations. State space models osit a world in which the true state cannot be measured directly but only inferred frmo what can be measured. State space models also rely on specifying the dynamics of a system such as how the true state of the world evolves over time both due to internal dynamics and the external forces that are applied to a system.

Three main applications : 

- Filtering : way of deciding how to weigh the most recent information against past information in updating our estimate of state
- Forecasting is the prediction of the future state without any information about the future.
- Smoothing is the use of future and past information in making a best estimate of the state at a given time.

## Kalman Filter

Mathematical formulation : 

$$ x_t = F \times x_{t-1} + B \times u_{t} + w_{t} $$
$$ y_t = A \times x_t + v_t $$
$$\hat{x}_t = K_{t} \times y_t + (1-K_t) \times \hat{x}_{t-1}$$

## Hiden Markov Models

Hidden Markov Model (HMM) is a statistical Markov model in which the system being modeled is assumed to be a Markov process – call it X  – with unobservable ("hidden") states. HMM assumes that there is another process Y  whose behavior "depends" on  X. The goal is to learn about X by observing Y  Y. HMM stipulates that, for each time instance n}, the conditional probability distribution of Y n given the history { X n = x n } n ≤ n0  must not depend on { x n } n < n 0 . 

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

from hmmlearn import hmm

##############################################################
# Prepare parameters for a 4-components HMM
# Initial population probability
startprob = np.array([0.6, 0.3, 0.1, 0.0])
# The transition matrix, note that there are no transitions possible
# between component 1 and 3
transmat = np.array([[0.7, 0.2, 0.0, 0.1],
                     [0.3, 0.5, 0.2, 0.0],
                     [0.0, 0.3, 0.5, 0.2],
                     [0.2, 0.0, 0.2, 0.6]])
# The means of each component
means = np.array([[0.0,  0.0],
                  [0.0, 11.0],
                  [9.0, 10.0],
                  [11.0, -1.0]])
# The covariance of each component
covars = .5 * np.tile(np.identity(2), (4, 1, 1))

# Build an HMM instance and set parameters
model = hmm.GaussianHMM(n_components=4, covariance_type="full")

# Instead of fitting it from the data, we directly set the estimated
# parameters, the means and covariance of the components
model.startprob_ = startprob
model.transmat_ = transmat
model.means_ = means
model.covars_ = covars
###############################################################

# Generate samples
X, Z = model.sample(500)

# Plot the sampled data
plt.plot(X[:, 0], X[:, 1], ".-", label="observations", ms=6,
         mfc="orange", alpha=0.7)

# Indicate the component numbers
for i, m in enumerate(means):
    plt.text(m[0], m[1], 'Component %i' % (i + 1),
             size=17, horizontalalignment='center',
             bbox=dict(alpha=.7, facecolor='w'))
plt.legend(loc='best')
plt.show()

# Generating and Selecting Features for Time Series

Use of tsfresh python module

In [ ]:
df_ts, y  = load_robot_execution_failures()
df_ts

In [ ]:
extracted_features = extract_features(df_ts, column_id="id", column_sort="time")
extracted_features

In [ ]:
features_filtered_direct = extract_relevant_features(df_ts, y,column_id='id', column_sort='time')
features_filtered_direct

# Machine Learning for Time Series

- Pretty similar to cross-sectional Machine Learning
- xgboost seems extremely effective with lightBM

Distances metrics used for Time Series : 
- Fréchet Distance : macimum distance between two curves during a time warping like traversal of the curves that always seeks to minimize the distance between two curves
- Pearson correlation 
- Longest common subsesequence
- Dynamic Time Warping (DTW)

![title](data_general/images/Euclidean-distance-vs-DTW.png)

In [ ]:
dstock = pd.read_csv('data_general/stocks.csv', sep=',', index_col='Date')
dstock.index = pd.to_datetime(dstock.index)

km_dba = TimeSeriesKMeans(
    n_clusters=5,
    metric="dtw",
    max_iter=5,
    max_iter_barycenter=5,
    random_state=0).fit(dstock.values)
dstock['cluster'] = km_dba.predict(dstock.values)
sns.countplot(x='cluster',data=dstock)

In [ ]:
sns.jointplot(data=dstock, x="AAPL", y="AMZN",  hue='cluster')